# Detecção de tumor cerebral utilizando modelo de neural network

Este modelo de rede neural convolutional tem como objetivo detectar a presença de um tumor cerebral.

In [6]:
import numpy as np
import os
import cv2
import random
import pickle
from zipfile import ZipFile

import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
import pickle
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
!ls 'drive/My Drive/Brain/brain-mri-tumor-detection-dataset'

NO  YES


# Setup da aplicação

In [50]:
os.environ['PYTHONHASHSEED']=str(66)
tf.random.set_seed(66)
np.random.seed(66)
random.seed(66)

categories = ['NO', 'YES']

# Preparação dos dados

In [9]:
def prepare_data(datadir, img_size=28):
	file_list = []
	class_list = []
	x = []
	y = []
	data = []
	error = False
	categories = ['NO', 'YES']

	for category in categories: 
		path = os.path.join(datadir, category)
		class_index = categories.index(category)
		for img in os.listdir(path):
			try:
				img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
				new_array = cv2.resize(img_array, (img_size, img_size))
				data.append([new_array, class_index])
			except Exception as e:
				error = True
				pass

	random.shuffle(data)

	for features, label in data:
		x.append(features)
		y.append(label)

	x = np.array(x).reshape(-1, img_size, img_size, 1)
	if error:
		print('Erro ao processar algums imagens')
	else:
		print('Imagens processadas com sucesso')
	return x, y

In [10]:
x, y = prepare_data('drive/My Drive/Brain/brain-mri-tumor-detection-dataset')

Imagens processadas com sucesso


In [11]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=5)

In [12]:
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.1, random_state=5)

In [16]:
y_train = np.array(y_train).reshape(-1, 1)
y_test = np.array(y_test).reshape(-1, 1)
y_val = np.array(y_val).reshape(-1, 1)

x_train = x_train / 255
x_test = x_test / 255
x_val = x_val / 255

# Definição do modelo e explicação

In [17]:
model = Sequential()

# convolutional layers
model.add(Conv2D(32, (3, 3), input_shape = x_train.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# hidden layers
model.add(Flatten())
model.add(Dense(128))
model.add(Activation("relu"))

model.add(Dense(128))
model.add(Activation("relu"))

# output layer
model.add(Dense(2))
model.add(Activation("softmax"))

# Compile
model.compile(loss="sparse_categorical_crossentropy",
      optimizer="adam",
      metrics=["accuracy"])

## Treino do modelo

In [18]:
model.fit(x_train, y_train, batch_size=128, epochs=100, verbose=1)

Epoch 1/100
2/2 [==============================] - 0s 23ms/step - loss: 0.6970 - accuracy: 0.4915
Epoch 2/100
2/2 [==============================] - 0s 22ms/step - loss: 0.6743 - accuracy: 0.5819
Epoch 3/100
2/2 [==============================] - 0s 21ms/step - loss: 0.6730 - accuracy: 0.5819
Epoch 4/100
2/2 [==============================] - 0s 21ms/step - loss: 0.6737 - accuracy: 0.5819
Epoch 5/100
2/2 [==============================] - 0s 29ms/step - loss: 0.6647 - accuracy: 0.5819
Epoch 6/100
2/2 [==============================] - 0s 22ms/step - loss: 0.6680 - accuracy: 0.5819
Epoch 7/100
2/2 [==============================] - 0s 22ms/step - loss: 0.6599 - accuracy: 0.5819
Epoch 8/100
2/2 [==============================] - 0s 21ms/step - loss: 0.6623 - accuracy: 0.5819
Epoch 9/100
2/2 [==============================] - 0s 21ms/step - loss: 0.6570 - accuracy: 0.5819
Epoch 10/100
2/2 [==============================] - 0s 21ms/step - loss: 0.6473 - accuracy: 0.5819
Epoch 11/100
2/2 [=

In [20]:
model.save('brain_tumor_model.hdf5')

In [26]:
!cp brain_tumor_model.hdf5 drive/My\ Drive/Brain/Models

# Teste e evaluation do modelo

In [41]:
scores = model.evaluate(x_test, y_test)
print("test length ", len(x_test))
print("test score %.2f%%" % (np.mean(scores[1] * 100)))

3/3 [==============================] - 0s 5ms/step - loss: 0.4025 - accuracy: 0.8676
test length  68
test score 86.76%


In [43]:
scores = model.evaluate(x_val, y_val)
print("test length ", len(x_val))
print("evaluation score %.2f%%" % (np.mean(scores[1] * 100)))

1/1 [==============================] - 0s 1ms/step - loss: 0.7894 - accuracy: 0.8750
test length  8
evaluation score 87.50%


# Detectando o tumor

In [51]:
def predict_image(file):
  img_array = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
  new_array = cv2.resize(img_array, (28, 28))
  img = new_array.reshape(-1, 28, 28, 1)
  prediction = model.predict([img])
  prediction = list(prediction[0])
  print(categories[prediction.index(max(prediction))])

In [52]:
predict_image('drive/My Drive/Brain/brain-mri-tumor-detection-dataset/YES/Y21.jpg')

YES


In [53]:
predict_image('drive/My Drive/Brain/brain-mri-tumor-detection-dataset/NO/N21.jpg')

NO
